In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVC
from sklearn.decomposition import PCA

In [6]:
#Import data
listings_X = pd.read_csv("data/listings.csv")
calendar = pd.read_csv("data/calendar.csv")
neighborhoods = pd.read_csv("data/neighbourhoods.csv")

In [10]:
#Preprocessing


listings_Y = listings_X['price']
listsings_X = listings_X.drop(['price'], axis=1)

#generate any classes desired from price ranges potentially

In [18]:
#Model
lnr = LinearRegression()
lgr = LogisticRegression()
svm = SVC()

In [ ]:
#Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(listings_X, listings_Y, test_size=0.20)

In [8]:
#Training

In [9]:
#Testing

In [ ]:
#Evalutation/Graphs